In [1]:
import time
import cv2
import os
import datetime
import csv

import serial
from threading import Thread

In [3]:
# 비디오 들어오나 테스트
cap=cv2.VideoCapture(0)
if not cap.isOpened():
    print("error")
    exit()
print(cap.get(cv2.CAP_PROP_FPS))
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
while(True):
    ret,cam=cap.read()
    if ret==True:
        cv2.imshow("cam",cam)
        if cv2.waitKey(1) & 0xFF == 27: # esc 키를 누르면 닫음
            cv2.destroyAllWindows()
            cap.release()
            break

30.0
-1.0


# 웹캠으로 프레임 캡처하기

In [6]:
def video():
    init_time = 0
    cap = cv2.VideoCapture(0)
    print(cap.get(cv2.CAP_PROP_FPS))
    print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = 30
    imgs=[]
    msec=[]
    while(cap.isOpened()):
            ret,frame=cap.read()
            
            if ret ==True :
                cv2.imshow('frame', frame)
                if init_time==0:
                    init_time=time.time()
                    init_frame=cap.get(cv2.CAP_PROP_POS_MSEC)
                curr_time = datetime.datetime.now()
                curr_frame=cap.get(cv2.CAP_PROP_POS_MSEC)-init_frame
                print("프레임위치",curr_frame,"현재시간",datetime.datetime.now())# 시작부터 경과시간(msec)
                
                if curr_frame>3000:
                    cap.release() # 멈추기
                    cv2.destroyAllWindows()
                    break
                imgs.append(frame)
                msec.append(curr_frame*0.001)
                
            else :
                cap.release() # 멈추기
                cv2.destroyAllWindows()
                print("error")
                break
            if cv2.waitKey(1) & 0xFF == 27: # esc 키를 누르면 닫음
                cv2.destroyAllWindows()
                cap.release()
    print(len(imgs))
    
    
    
    count=1
    for i in range(len(imgs)):
        cv2.imwrite(images_path+'frame_%d_%s.png'%(count,msec[i]), imgs[i], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        count=count+1

            

In [11]:
path_num = 100
images_path ='D:\\prlab\\ysg\\rppg\\new\\/images/%d/'%path_num 
if not os.path.isdir(images_path):   # 폴더 있는지 확인하고 없으면 생성   
    print("no dir")
    os.mkdir(images_path)
video()

no dir
30.0
-1.0
프레임위치 0.0 현재시간 2021-04-07 14:30:54.663622
프레임위치 33.33329999446869 현재시간 2021-04-07 14:30:54.687474
프레임위치 66.66670000553131 현재시간 2021-04-07 14:30:54.718981
프레임위치 100.0 현재시간 2021-04-07 14:30:54.751828
프레임위치 133.3332999944687 현재시간 2021-04-07 14:30:54.784170
프레임위치 166.6667000055313 현재시간 2021-04-07 14:30:54.817106
프레임위치 200.0 현재시간 2021-04-07 14:30:54.849879
프레임위치 233.3332999944687 현재시간 2021-04-07 14:30:54.882239
프레임위치 266.6667000055313 현재시간 2021-04-07 14:30:54.914319
프레임위치 300.0 현재시간 2021-04-07 14:30:54.944850
프레임위치 333.3332999944687 현재시간 2021-04-07 14:30:54.976784
프레임위치 366.6667000055313 현재시간 2021-04-07 14:30:55.023610
프레임위치 400.0 현재시간 2021-04-07 14:30:55.057890
프레임위치 433.3332999944687 현재시간 2021-04-07 14:30:55.089480
프레임위치 466.6667000055313 현재시간 2021-04-07 14:30:55.120133
프레임위치 500.0 현재시간 2021-04-07 14:30:55.151713
프레임위치 533.3332999944687 현재시간 2021-04-07 14:30:55.184108
프레임위치 566.6667000055313 현재시간 2021-04-07 14:30:55.216221
프레임위치 600.0 현재시간 2021-04-07 14:30:55.247745
프레임위치

# ppg 측정 연동
- pyserial 연결은 관리자모드의 cmd여야 connection가능

In [4]:


class Ubpulse:
    '''
    https://github.com/ubpulse/ubpulse-360-340-320/blob/master/LXD23_ubpulse3xx_CommunicationSpec.pdf
    '''
    def __init__(self, serialPort, serialBaud, dataNumBytes):
        self.isConnected = False
        self.port = serialPort
        self.baud = 115200
        self.dataNumBytes = min(4096, dataNumBytes)
        self.rawData = bytearray(dataNumBytes)
        self.isRun = True
        self.isReceiving = False
        self.thread = None

        self.sync_after = False
        self.packet_tx_idx = 0
        self.data_prev = 0

        self.PUD0 = 0
        self.PCDT_PUD2_CRD = 0
        self.PC = 0  # PacketCount
        self.PUD1 = 0
        self.PCD = 0  # PacketCyclicData

        self.ppg_upper = 0
        self.ppg_lower = 0
        self.pi_upper = 0
        self.pi_lower = 0

        self.ComDeviceID = 0
        self.ppg = []
        self.real_time = []
        self.ppi = 0
        self.bpm = 0
        self.pi = 0

        print('Trying to connect to: ' + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
        try:
            self.serialConnection = serial.Serial(serialPort, serialBaud, timeout=1/256)
            self.isConnected = True
            print('Connected to ' + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
        except:
            print("Failed to connect with " + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
            raise Exception("Connection failed")

    def readSerialStart(self):
        if isinstance(self.thread, type(None)):
            self.thread = Thread(target=self.backgroundThread)
            self.thread.daemon = True
            self.thread.start()
            # Block till we start receiving values
            while not self.isReceiving:
                time.sleep(0.1)

    def backgroundThread(self):  # retrieve data
        time.sleep(1.0)  # give some buffer time for retrieving data
        self.serialConnection.reset_input_buffer()
        while self.isRun:
            self.serialConnection.readinto(self.rawData)
            self.isReceiving = True

            for i in range(len(self.rawData)):
                data_crnt = self.rawData[i]
                if self.data_prev == 255 and data_crnt == 254:
                    self.sync_after = True
                    self.packet_tx_idx = 0
                self.data_prev = data_crnt

                if self.sync_after:
                    self.packet_tx_idx += 1

                    if self.packet_tx_idx > 1:
                        if self.packet_tx_idx == 2:
                            self.PUD0 = data_crnt
                            self.ppi = (self.PUD1 & 0x07) * 256 + self.PUD0
                        elif self.packet_tx_idx == 3:
                            self.PCDT_PUD2_CRD = data_crnt
                        elif self.packet_tx_idx == 4:
                            self.PC = data_crnt
                        elif self.packet_tx_idx == 5:
                            self.PUD1 = data_crnt
                        elif self.packet_tx_idx == 6:
                            self.PCD = data_crnt
                            if self.PC == 2:
                                self.bpm = self.PCD
                            elif self.PC == 3:
                                self.pi_upper = self.PCD
                            elif self.PC == 4:
                                self.pi_lower = self.PCD
                                self.pi = ((self.pi_upper & 0x0F) * 256 + self.pi_lower) * 0.01
                            elif self.PC == 30:
                                self.ComDeviceID = self.PCD
                        elif self.packet_tx_idx == 7:
                            self.ppg_upper = data_crnt
                        elif self.packet_tx_idx == 8:
                            self.ppg_lower = data_crnt
                            self.ppg.append((self.ppg_upper & 0x0F) * 256 + self.ppg_lower)
                            self.real_time.append(str(datetime.datetime.now()))
                        elif self.packet_tx_idx > 8:
                            self.sync_after = False

    def flush(self):
        self.serialConnection.flush()

    def close(self):
        self.isRun = False
        self.serialConnection.close()
        print('Disconnected...')


def video(serial_port,images_path,cppg_path,timeout):
    init_time = 0
    cap = cv2.VideoCapture(0)
    print(cap.get(cv2.CAP_PROP_FPS))
    print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = 30
    imgs=[]
    rppg_msec=[]
    serial_port.readSerialStart() # 시작 시 cppg기록도 시작(background thread시작)
    while(cap.isOpened()):
            ret,frame=cap.read()
            
            if ret ==True :
                cv2.imshow('frame', frame)
                if init_time==0:
                    init_time=time.time()
                    init_frame=cap.get(cv2.CAP_PROP_POS_MSEC)
                    
                    
                curr_time = datetime.datetime.now()
                curr_frame=cap.get(cv2.CAP_PROP_POS_MSEC)-init_frame
                print("프레임위치(sec)",curr_frame*0.001,"현재시간",datetime.datetime.now())# 시작부터 경과시간(msec)
                
                if curr_frame>(timeout*1000):
                    serial_port.close() # cppg기록 stop
                    cap.release() # 멈추기
                    cv2.destroyAllWindows()
                    break
                imgs.append(frame)
                rppg_msec.append(curr_frame)
                
            else :
                serial_port.close()
                cap.release() # 멈추기
                cv2.destroyAllWindows()
                print("error")
                break
            if cv2.waitKey(1) & 0xFF == 27: # esc 키를 누르면 닫음
                serial_port.close()
                cv2.destroyAllWindows()
                cap.release()
    print(len(imgs))
    
    
    # rppg img저장
    count=1
    for i in range(len(imgs)):
        cv2.imwrite(images_path+'frame_%d_%s.png'%(count,rppg_msec[i]), imgs[i], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        print("save",images_path,'frame_%d_%s.png'%(count,rppg_msec[i]))
        count=count+1
    
    # cppg pulse.csv저장
    with open(cppg_path, 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(serial_port.ppg)
        wr.writerow(serial_port.real_time)
        f.close()

def cppg_test(serial_port, cppg_path, timeout):
    init_time = 0
    curr_time = 0
    while True:
        if curr_time == 0:
            init_time = time.time()
            serial_port.readSerialStart()  # starts background thread
            
        curr_time = time.time()
        
        if curr_time - init_time > timeout:
            serial_port.serialConnection.close()
            print(curr_time - init_time)
            break
    print(serial_port.real_time) 
    with open(cppg_path, 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(serial_port.ppg)
        wr.writerow(serial_port.real_time)
        f.close()


if __name__ == '__main__':
    path_num =5 
    cppg_path = "D:\\prlab\\ysg\\rppg\\new\\data\\cppg\\cppg%d.csv"%path_num ##ppg신호 저장할 경로
    images_path ='D:\\prlab\\ysg\\rppg\\new\\images\\%d/'%path_num 
    if not os.path.isdir(images_path):
        print("no dir")# 폴더 있는지 확인하고 없으면 생성                                                   
        os.mkdir(images_path)
    
    portName = 'COM3'
    baudRate = 9600 # 115200
    dataNumBytes = 8  # number of bytes of 1 data point
    sp = Ubpulse(portName, baudRate, dataNumBytes)  # initializes all required variables
    
    
    video(sp,images_path,cppg_path,660) # timeout은 sec
    #cppg_test(sp,cppg_path,timeout=10)
    #cppg_video_test(sp, cppg_path, images_path)

no dir
Trying to connect to: COM3 at 9600 BAUD.
Connected to COM3 at 9600 BAUD.
30.0
-1.0
프레임위치(sec) 0.0 현재시간 2021-04-12 16:13:26.940274
프레임위치(sec) 0.033333300000056625 현재시간 2021-04-12 16:13:26.952548
프레임위치(sec) 0.06666669999994337 현재시간 2021-04-12 16:13:26.981845
프레임위치(sec) 0.1 현재시간 2021-04-12 16:13:27.014658
프레임위치(sec) 0.13333330000005664 현재시간 2021-04-12 16:13:27.048406
프레임위치(sec) 0.16666669999994338 현재시간 2021-04-12 16:13:27.076891
프레임위치(sec) 0.2 현재시간 2021-04-12 16:13:27.109129
프레임위치(sec) 0.23333330000005664 현재시간 2021-04-12 16:13:27.142247
프레임위치(sec) 0.26666669999994336 현재시간 2021-04-12 16:13:27.172234
프레임위치(sec) 0.3 현재시간 2021-04-12 16:13:27.204829
프레임위치(sec) 0.3333333000000566 현재시간 2021-04-12 16:13:27.236735
프레임위치(sec) 0.3666666999999434 현재시간 2021-04-12 16:13:27.284576
프레임위치(sec) 0.4 현재시간 2021-04-12 16:13:27.317568
프레임위치(sec) 0.43333330000005665 현재시간 2021-04-12 16:13:27.348982
프레임위치(sec) 0.46666669999994337 현재시간 2021-04-12 16:13:27.380994
프레임위치(sec) 0.5 현재시간 2021-04-12 16:13:27.412870

프레임위치(sec) 4.7 현재시간 2021-04-12 16:13:31.604669
프레임위치(sec) 4.733333300000057 현재시간 2021-04-12 16:13:31.638981
프레임위치(sec) 4.766666699999943 현재시간 2021-04-12 16:13:31.686449
프레임위치(sec) 4.8 현재시간 2021-04-12 16:13:31.720229
프레임위치(sec) 4.8333333000000565 현재시간 2021-04-12 16:13:31.751161
프레임위치(sec) 4.866666699999944 현재시간 2021-04-12 16:13:31.782909
프레임위치(sec) 4.9 현재시간 2021-04-12 16:13:31.815210
프레임위치(sec) 4.933333300000057 현재시간 2021-04-12 16:13:31.846693
프레임위치(sec) 4.966666699999943 현재시간 2021-04-12 16:13:31.879310
프레임위치(sec) 5.0 현재시간 2021-04-12 16:13:31.908710
프레임위치(sec) 5.033333300000057 현재시간 2021-04-12 16:13:31.941443
프레임위치(sec) 5.066666699999944 현재시간 2021-04-12 16:13:31.974934
프레임위치(sec) 5.1000000000000005 현재시간 2021-04-12 16:13:32.007415
프레임위치(sec) 5.133333300000057 현재시간 2021-04-12 16:13:32.037611
프레임위치(sec) 5.1666666999999435 현재시간 2021-04-12 16:13:32.084209
프레임위치(sec) 5.2 현재시간 2021-04-12 16:13:32.117778
프레임위치(sec) 5.233333300000057 현재시간 2021-04-12 16:13:32.150593
프레임위치(sec) 5.266666699999943 현

프레임위치(sec) 9.566666699999944 현재시간 2021-04-12 16:13:36.485064
프레임위치(sec) 9.6 현재시간 2021-04-12 16:13:36.517848
프레임위치(sec) 9.633333300000057 현재시간 2021-04-12 16:13:36.550430
프레임위치(sec) 9.666666699999944 현재시간 2021-04-12 16:13:36.581145
프레임위치(sec) 9.700000000000001 현재시간 2021-04-12 16:13:36.612066
프레임위치(sec) 9.733333300000057 현재시간 2021-04-12 16:13:36.643974
프레임위치(sec) 9.766666699999943 현재시간 2021-04-12 16:13:36.676525
프레임위치(sec) 9.8 현재시간 2021-04-12 16:13:36.709327
프레임위치(sec) 9.833333300000056 현재시간 2021-04-12 16:13:36.740776
프레임위치(sec) 9.866666699999943 현재시간 2021-04-12 16:13:36.771989
프레임위치(sec) 9.9 현재시간 2021-04-12 16:13:36.804126
프레임위치(sec) 9.933333300000056 현재시간 2021-04-12 16:13:36.836257
프레임위치(sec) 9.966666699999944 현재시간 2021-04-12 16:13:36.885039
프레임위치(sec) 10.0 현재시간 2021-04-12 16:13:36.916418
프레임위치(sec) 10.033333300000058 현재시간 2021-04-12 16:13:36.947832
프레임위치(sec) 10.066666699999944 현재시간 2021-04-12 16:13:36.980624
프레임위치(sec) 10.1 현재시간 2021-04-12 16:13:37.012400
프레임위치(sec) 10.133333300000057

프레임위치(sec) 15.700000000000001 현재시간 2021-04-12 16:13:42.613018
프레임위치(sec) 15.733333300000057 현재시간 2021-04-12 16:13:42.644662
프레임위치(sec) 15.766666699999943 현재시간 2021-04-12 16:13:42.676688
프레임위치(sec) 15.8 현재시간 2021-04-12 16:13:42.708756
프레임위치(sec) 15.833333300000056 현재시간 2021-04-12 16:13:42.740676
프레임위치(sec) 15.866666699999943 현재시간 2021-04-12 16:13:42.772296
프레임위치(sec) 15.9 현재시간 2021-04-12 16:13:42.804920
프레임위치(sec) 15.933333300000056 현재시간 2021-04-12 16:13:42.837172
프레임위치(sec) 15.966666699999944 현재시간 2021-04-12 16:13:42.883872
프레임위치(sec) 16.0 현재시간 2021-04-12 16:13:42.916471
프레임위치(sec) 16.033333300000056 현재시간 2021-04-12 16:13:42.949511
프레임위치(sec) 16.066666699999942 현재시간 2021-04-12 16:13:42.980309
프레임위치(sec) 16.1 현재시간 2021-04-12 16:13:43.012494
프레임위치(sec) 16.133333300000057 현재시간 2021-04-12 16:13:43.044164
프레임위치(sec) 16.166666699999944 현재시간 2021-04-12 16:13:43.076186
프레임위치(sec) 16.2 현재시간 2021-04-12 16:13:43.107908
프레임위치(sec) 16.23333330000006 현재시간 2021-04-12 16:13:43.140418
프레임위치(sec) 16.266

프레임위치(sec) 20.566666699999942 현재시간 2021-04-12 16:13:47.476636
프레임위치(sec) 20.6 현재시간 2021-04-12 16:13:47.508896
프레임위치(sec) 20.633333300000057 현재시간 2021-04-12 16:13:47.540953
프레임위치(sec) 20.666666699999944 현재시간 2021-04-12 16:13:47.571846
프레임위치(sec) 20.7 현재시간 2021-04-12 16:13:47.604651
프레임위치(sec) 20.73333330000006 현재시간 2021-04-12 16:13:47.636499
프레임위치(sec) 20.766666699999945 현재시간 2021-04-12 16:13:47.684674
프레임위치(sec) 20.8 현재시간 2021-04-12 16:13:47.716558
프레임위치(sec) 20.833333300000056 현재시간 2021-04-12 16:13:47.748642
프레임위치(sec) 20.866666699999943 현재시간 2021-04-12 16:13:47.781181
프레임위치(sec) 20.900000000000002 현재시간 2021-04-12 16:13:47.812172
프레임위치(sec) 20.933333300000058 현재시간 2021-04-12 16:13:47.844933
프레임위치(sec) 20.966666699999944 현재시간 2021-04-12 16:13:47.876698
프레임위치(sec) 21.0 현재시간 2021-04-12 16:13:47.908339
프레임위치(sec) 21.033333300000056 현재시간 2021-04-12 16:13:47.940804
프레임위치(sec) 21.066666699999942 현재시간 2021-04-12 16:13:47.972340
프레임위치(sec) 21.1 현재시간 2021-04-12 16:13:48.004699
프레임위치(sec) 21.133

...

save D:\prlab\ysg\rppg\new\images\5/ frame_19139_639233.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19140_639266.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19141_639300.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19142_639333.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19143_639366.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19144_639400.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19145_639433.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19146_639466.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19147_639500.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19148_639533.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19149_639566.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19150_639600.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19151_639633.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19152_639666.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19153_639700.0.png
save D:\prlab\ysg\rppg\new\images\5/ fra

save D:\prlab\ysg\rppg\new\images\5/ frame_19268_643533.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19269_643566.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19270_643600.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19271_643633.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19272_643666.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19273_643700.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19274_643733.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19275_643766.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19276_643800.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19277_643833.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19278_643866.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19279_643900.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19280_643933.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19281_643966.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19282_644000.0.png
save D:\prlab\ysg\rppg\new\images\5/ fra

save D:\prlab\ysg\rppg\new\images\5/ frame_19397_647833.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19398_647866.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19399_647900.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19400_647933.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19401_647966.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19402_648000.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19403_648033.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19404_648066.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19405_648100.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19406_648133.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19407_648166.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19408_648200.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19409_648233.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19410_648266.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19411_648300.0.png
save D:\prlab\ysg\rppg\new\images\5/ fra

save D:\prlab\ysg\rppg\new\images\5/ frame_19526_652133.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19527_652166.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19528_652200.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19529_652233.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19530_652266.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19531_652300.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19532_652333.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19533_652366.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19534_652400.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19535_652433.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19536_652466.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19537_652500.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19538_652533.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19539_652566.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19540_652600.0.png
save D:\prlab\ysg\rppg\new\images\5/ fra

save D:\prlab\ysg\rppg\new\images\5/ frame_19655_656433.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19656_656466.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19657_656500.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19658_656533.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19659_656566.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19660_656600.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19661_656633.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19662_656666.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19663_656700.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19664_656733.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19665_656766.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19666_656800.0.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19667_656833.3333.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19668_656866.6667.png
save D:\prlab\ysg\rppg\new\images\5/ frame_19669_656900.0.png
save D:\prlab\ysg\rppg\new\images\5/ fra

In [3]:
ser=serial.Serial('COM3', 9600, timeout=1/256)
ser.close()